In [125]:
import pandas as pd
import numpy as np
import csv
import nltk

nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     /Users/jamesseppi/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [126]:
# Load data from CSVs

# for whatever reason, 'latin-1' needs to be used as the encoding on these files,
# otherwise a Unicode error is thrown
default_encoding = 'latin-1'

PARKED_CARTS_DF = pd.read_csv('data/18f_parked_carts.csv', encoding=default_encoding)
PARKED_ITEMS_DF = pd.read_csv('data/18f_parked_items.csv', encoding=default_encoding)
CHECKOUTS_DF = pd.read_csv('data/18f_checkouts.csv', encoding=default_encoding)

# TODO: what about 18f_checkout_items2.csv ?
CHECKOUT_ITEMS_DF = pd.read_csv('data/18f_checkout_items.csv', encoding=default_encoding)


In [127]:
# Total number of parked items
len(PARKED_ITEMS_DF)

320747

In [128]:
# Total number of checkout items
len(CHECKOUT_ITEMS_DF)

419671

In [129]:
# Display the columns in CHECKOUT_ITEMS_DF
list(CHECKOUT_ITEMS_DF)

['sessionid',
 'user_name',
 'action_time',
 'agency',
 'event_type',
 'order_session_number',
 'bpa_number',
 'ct',
 'mfr',
 'part',
 'product_name',
 'search_query',
 'unit_price',
 'quantity',
 'uom',
 'payment_type']

In [130]:
# Display the columns in PARKED_ITEMS_DF
list(PARKED_ITEMS_DF)

['park_cart_id',
 'bpa_number',
 'ct',
 'event_type',
 'mfr',
 'part',
 'product_name',
 'search_query',
 'unit_price',
 'quantity',
 'uom']

In [131]:
# How many items were purchased from a direct link or browse, ie, an empty `search_query` field?
checkout_items_without_search_query = CHECKOUT_ITEMS_DF[CHECKOUT_ITEMS_DF['search_query'].isnull()]
len(checkout_items_without_search_query)

220649

In [132]:
# ...as a percentage
print("{0:.2f}%".format(len(checkout_items_without_search_query) / len(CHECKOUT_ITEMS_DF) * 100.0))

52.58%


In [133]:
# Annotate item dataframes with a list of search terms for each row

def search_query_to_list(val):
    # val is a string that looks like:
    # "['K40', 'CONTACT', 'POSITIONER']"
    #
    # Each val starts and ends with '[' and ']',
    # so strip those off
    if pd.isnull(val):
        # return empty list for empty cell valus
        return []
    val = val[1:len(val)-1]
    reader = csv.reader([val], quotechar="'", skipinitialspace=True)
    terms = list(reader)[0]
    return [t.strip() for t in terms]

def annotate_with_search_query_list(df):
    df['_search_query_list'] = df['search_query'].apply(search_query_to_list)


annotate_with_search_query_list(PARKED_ITEMS_DF)
annotate_with_search_query_list(CHECKOUT_ITEMS_DF)

In [134]:
# Find repeated search terms

def search_term_counts(df):
    term_counts = dict()
    for term_list in df['_search_query_list']:
        for term in term_list:
            term = term.lower()
            if term in term_counts:
                term_counts[term] += 1
            else:
                term_counts[term] = 1
    return pd.DataFrame(list(term_counts.items()), columns=['term', 'count'])         

# Get search term counts for checkout items
checkout_item_search_term_counts = search_term_counts(CHECKOUT_ITEMS_DF)

# Display the top 20 search terms of checkout item search terms
checkout_item_search_term_counts.sort_values(by=['count'], ascending=False)[:20]

,term,count
584,gsa copier paper hammermill,428
585,gsa print paper,428
269,varidesk,403
1884,aa batteries,289
338,7510-01-545-3763,287
577,7510-01-545-3753,262
227,7510015453763,249
4122,pens,240
2737,copy paper,234
1235,ham86700,234


In [135]:
# Get search term counts for parked items
parked_item_search_term_counts = search_term_counts(PARKED_ITEMS_DF)

# Display the top 20 search terms of parked item search terms
parked_item_search_term_counts.sort_values(by=['count'], ascending=False)[:20]

,term,count
701,aa batteries,326
1599,7510-01-545-3763,320
783,7510-01-545-3753,301
624,scissors,286
35,pens,285
403,aaa batteries,269
394,stapler,261
122,highlighters,258
238,7510015453763,246
778,black,245


In [175]:
# Get counts of rows with no search terms, part-number-like search terms, and word-like search terms

# Cache the corpus of words since grabbing it is a slow operation
# and convert it to a `set` for faster lookups.
CORPUS_WORDS = set(nltk.corpus.words.words())

def probably_not_a_word(s):
    '''
    Returns True if a string is not alphabetic or has
    any numbers in it.
    
    TODO: Might want a threshold, like at least 2 digits.
    '''
    return s and not s.isalpha()

def probably_has_part_number(term_list):
    '''
    Returns True if any term in the given list is
    probably not a word.
    '''
    for term in term_list:
        for t in term.split():
            if probably_not_a_word(t):
                return True
    return False

def is_word(s):
    '''
    Returns True if the given string is at least 2 chars long,
    is only alphabetic, and is in the nltk corpus of words.
    '''
    s = s.lower()
    return len(s) > 2 and s.isalpha() and s in CORPUS_WORDS

def has_english_word(term_list):
    '''
    Checks if a list of search terms has an English word.
    '''
    for term in term_list:
        for t in term.split():
            if is_word(t):
                return True
    return False


def analyze_search_terms(df):
    rows_no_search = df[df['_search_query_list'].map(len) == 0]
    rows_with_search = df[df['_search_query_list'].map(len) > 0]
    rows_with_part_number_search = df[df['_search_query_list'].map(probably_has_part_number)]
    rows_english_search = df[df['_search_query_list'].map(has_english_word) == True]
    
    d = {
        'Total rows': len(df),
        'Rows with no search term': len(rows_no_search),
        'Rows with search terms': len(rows_with_search),
        'Rows with probable part number search': len(rows_with_part_number_search),
        'Rows with English word search': len(rows_english_search),
    }

    return pd.DataFrame(list(d.items()), columns=['label', 'count']) 


In [178]:
# Rough count of Checkout item search term counts
analyze_search_terms(CHECKOUT_ITEMS_DF)

,label,count
0,Total rows,419671
1,Rows with no search term,220649
2,Rows with search terms,199022
3,Rows with probable part number search,161537
4,Rows with English word search,64503


In [177]:
# Rough count of Parked item search term counts
analyze_search_terms(PARKED_ITEMS_DF)

,label,count
0,Total rows,320747
1,Rows with no search term,167741
2,Rows with search terms,153006
3,Rows with probable part number search,105339
4,Rows with English word search,70728


In [ ]:
# TODO: Instead of using exact queries, what about segmenting by words or lexemes?
# ie, finding counts for all terms that are like "pen" - "red pens", "ballpoint pens", "pen", etc

In [ ]:
# TODO: What's the average price of sold items?


In [ ]:
# TODO: What's the average price of parked items?
